<a href="https://colab.research.google.com/github/shahchhatru/AI_colab_notebooks/blob/main/SkipgramAndDeepWalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## this is an practice implementation of skip gram algorithmn


In [2]:
import numpy as np

In [3]:
CONTEXT_SIZE=2


In [5]:
text="""
Nature, a tranquil canvas, paints landscapes with vibrant hues, whispering tales
through rustling leaves and dancing streams. Life blooms in meadows, and mountains
echo with ancient stories. Sunsets bathe the world in golden warmth, while moonlit nights weave dreams.
Breezes carry fragrances, and rivers sing timeless melodies. In this harmonious symphony, every creature finds its rhythm,
connecting to the primal heartbeat of Earth. Nature, a sanctuary of simplicity, invites us to pause,
breathe, and embrace the beauty woven into the fabric of existence.
""".split()

In [6]:
text

['Nature,',
 'a',
 'tranquil',
 'canvas,',
 'paints',
 'landscapes',
 'with',
 'vibrant',
 'hues,',
 'whispering',
 'tales',
 'through',
 'rustling',
 'leaves',
 'and',
 'dancing',
 'streams.',
 'Life',
 'blooms',
 'in',
 'meadows,',
 'and',
 'mountains',
 'echo',
 'with',
 'ancient',
 'stories.',
 'Sunsets',
 'bathe',
 'the',
 'world',
 'in',
 'golden',
 'warmth,',
 'while',
 'moonlit',
 'nights',
 'weave',
 'dreams.',
 'Breezes',
 'carry',
 'fragrances,',
 'and',
 'rivers',
 'sing',
 'timeless',
 'melodies.',
 'In',
 'this',
 'harmonious',
 'symphony,',
 'every',
 'creature',
 'finds',
 'its',
 'rhythm,',
 'connecting',
 'to',
 'the',
 'primal',
 'heartbeat',
 'of',
 'Earth.',
 'Nature,',
 'a',
 'sanctuary',
 'of',
 'simplicity,',
 'invites',
 'us',
 'to',
 'pause,',
 'breathe,',
 'and',
 'embrace',
 'the',
 'beauty',
 'woven',
 'into',
 'the',
 'fabric',
 'of',
 'existence.']

In [7]:
skipgrams =[]
for i in range(CONTEXT_SIZE,len(text)-CONTEXT_SIZE):
  array=[text[j] for j in np.arange(i-CONTEXT_SIZE,i+CONTEXT_SIZE+1) if j!=i]
  skipgrams.append((text[i],array))

In [8]:
skipgrams

[('tranquil', ['Nature,', 'a', 'canvas,', 'paints']),
 ('canvas,', ['a', 'tranquil', 'paints', 'landscapes']),
 ('paints', ['tranquil', 'canvas,', 'landscapes', 'with']),
 ('landscapes', ['canvas,', 'paints', 'with', 'vibrant']),
 ('with', ['paints', 'landscapes', 'vibrant', 'hues,']),
 ('vibrant', ['landscapes', 'with', 'hues,', 'whispering']),
 ('hues,', ['with', 'vibrant', 'whispering', 'tales']),
 ('whispering', ['vibrant', 'hues,', 'tales', 'through']),
 ('tales', ['hues,', 'whispering', 'through', 'rustling']),
 ('through', ['whispering', 'tales', 'rustling', 'leaves']),
 ('rustling', ['tales', 'through', 'leaves', 'and']),
 ('leaves', ['through', 'rustling', 'and', 'dancing']),
 ('and', ['rustling', 'leaves', 'dancing', 'streams.']),
 ('dancing', ['leaves', 'and', 'streams.', 'Life']),
 ('streams.', ['and', 'dancing', 'Life', 'blooms']),
 ('Life', ['dancing', 'streams.', 'blooms', 'in']),
 ('blooms', ['streams.', 'Life', 'in', 'meadows,']),
 ('in', ['Life', 'blooms', 'meadows,',

In [9]:
print(skipgrams[0:2])

[('tranquil', ['Nature,', 'a', 'canvas,', 'paints']), ('canvas,', ['a', 'tranquil', 'paints', 'landscapes'])]


In [11]:
##let's see the vocabulary
vocab=set(text)
VOCAB_SIZE=len(vocab)
print(f"Length of vocabulary is={VOCAB_SIZE}")

Length of vocabulary is=70


Now that we have our vocabulary there is one more parameter that we need to define N , the dimensionality of the word vectors . Typically this value is set between 100 to 1000. But because of our small datasets we will set it to 10

### Skip gram model
#### It is composed of only two layers



*   A projection layer with Weight matrix *W<sub>embd</sub>* : It takes one hot encoded vector as input and returns the corresponding N-dim word embeddings. It acts as a simple lookup table that stores embedding of predefined dimensionality.

*   A fully connected layer with Weight matrix *W<sub>output</sub>* : It takes the word embedding as input and Outputs |V| dim logits. A softmax function is applied to these predictions to turn into probabilities.



In [12]:
#we can implement this model using gensim library.


In [13]:
!pip install -qU genism

ERROR: Could not find a version that satisfies the requirement genism (from versions: none)
ERROR: No matching distribution found for genism


In [14]:
from gensim.models.word2vec import Word2Vec

In [15]:
model=Word2Vec([text],
               sg=1, #skip gram =1
               vector_size=10,
               min_count=0,
               window=2,
               seed=0
               )

In [16]:
print(f"shape of W_embed : {model.wv.vectors.shape}")

shape of W_embed : (70, 10)


In [17]:
print(model)

Word2Vec<vocab=70, vector_size=10, alpha=0.025>


In [22]:
## We train the model for 10 epochs.
model.train([text],total_examples=model.corpus_count,epochs=10)

(278, 830)

In [23]:
print("Word embeddings")
print(model.wv[0])

Word embeddings
[ 0.06910036  0.02826009  0.00365436 -0.04655457 -0.03654416 -0.08919194
 -0.08470739 -0.09637518 -0.06494602  0.06502254]


In [26]:
model.wv.most_similar('landscapes', topn=10)

[('meadows,', 0.6518340706825256),
 ('this', 0.6309268474578857),
 ('mountains', 0.5691123604774475),
 ('through', 0.5659627914428711),
 ('warmth,', 0.5342003107070923),
 ('in', 0.45626121759414673),
 ('us', 0.4455518424510956),
 ('nights', 0.4314277768135071),
 ('existence.', 0.38293105363845825),
 ('beauty', 0.3573189079761505)]

In [29]:
vector = model.wv['mountains']  # get numpy vector of a word

In [30]:
vector

array([ 0.06698663,  0.05173635, -0.01819627, -0.00056968, -0.0153881 ,
        0.00637346, -0.05383362,  0.05713658, -0.08436236, -0.01690745],
      dtype=float32)